In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from MLP_classifier import MultiClassClassifier
from torch.utils.data import DataLoader
import sys
sys.path.append(".")
from dataset import DeepFakeDatasetFastLoad, OOD
import torch.nn as nn
import torch
sys.path.append("../tools")
from constants import PATH_TO_DATA, SEED, PATH_TO_DATA4
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split

In [10]:
device = 1
model = MultiClassClassifier().cuda(device=device)

lr = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

data = DeepFakeDatasetFastLoad(PATH_TO_DATA4 + "df_34000.pt")

rng = torch.Generator().manual_seed(SEED)
train_data, test_data = random_split(data,[0.8,0.2],generator=rng)

train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader  = DataLoader(test_data,batch_size=len(test_data),shuffle=True)

model.train()

MultiClassClassifier(
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=18, bias=True)
  (act): ReLU()
)

## Train for multi-class classification

In [4]:
model.set_generators_maps(gen_to_int=data.gen_to_int,int_to_gen=data.int_to_gen)

In [5]:
n_epochs = 1000
for epoch in range(1,n_epochs+1):
    for idx, batch in enumerate(train_loader):
        # prediction and loss
        pred = model(batch["features"].cuda(device))
        loss = loss_fn(pred,batch["generator"].cuda(device))

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    loss, current = loss.item(), idx*batch_size + len(batch["features"])
    if epoch%10 == 0 and epoch > 0:
        print(f"loss: {loss:>7f}  [{epoch:>5d}/{n_epochs:>5d}]")

loss: 1.323991  [   10/ 1000]
loss: 0.864932  [   20/ 1000]
loss: 0.626108  [   30/ 1000]
loss: 0.667035  [   40/ 1000]
loss: 0.507460  [   50/ 1000]
loss: 0.511223  [   60/ 1000]
loss: 0.361411  [   70/ 1000]
loss: 0.360735  [   80/ 1000]
loss: 0.303903  [   90/ 1000]
loss: 0.345464  [  100/ 1000]
loss: 0.301309  [  110/ 1000]
loss: 0.485785  [  120/ 1000]
loss: 0.357442  [  130/ 1000]
loss: 0.358514  [  140/ 1000]
loss: 0.304572  [  150/ 1000]
loss: 0.179329  [  160/ 1000]
loss: 0.322844  [  170/ 1000]
loss: 0.231258  [  180/ 1000]
loss: 0.283462  [  190/ 1000]
loss: 0.269941  [  200/ 1000]
loss: 0.413469  [  210/ 1000]
loss: 0.202649  [  220/ 1000]
loss: 0.285321  [  230/ 1000]
loss: 0.258902  [  240/ 1000]
loss: 0.170175  [  250/ 1000]
loss: 0.169469  [  260/ 1000]
loss: 0.218348  [  270/ 1000]
loss: 0.129989  [  280/ 1000]
loss: 0.214313  [  290/ 1000]
loss: 0.178507  [  300/ 1000]
loss: 0.287334  [  310/ 1000]
loss: 0.083524  [  320/ 1000]
loss: 0.152904  [  330/ 1000]
loss: 0.14

## Test for binary classification

In [6]:
import torch.types

with torch.no_grad():
    model.eval()
    for e in test_loader:
        accuracy = model.get_model_accuracy_binary(features=e["features"],
                                                   true_labels=e["label"],
                                                   device="cuda:" + str(device))
print(accuracy)

0.9738234877586365


### Comparison with SVM

In [8]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

clf = OneVsOneClassifier(LinearSVC(dual="auto"))

In [9]:
train_loader_all = DataLoader(train_data,batch_size=len(train_data))
for e in train_loader_all:
    X_train = e["features"]
    gen_train = e["generator"]
    label_train = e["label"]
for e in test_loader:
    X_test = e["features"]
    gen_test = e["generator"]
    label_test = e["label"]

clf.fit(X_train, gen_train) # train on multi-class classification

OneVsOneClassifier(estimator=LinearSVC(dual='auto'))

In [10]:
import numpy as np
pred = data.class_to_label(clf.predict(X_test))
np.mean(label_test.numpy() == pred.numpy()) # binary classification performance

0.97

## Test on multi-class classification

### Neural Network

In [ ]:
import torch.types

with torch.no_grad():
    model.eval()
    for e in test_loader:
        accuracy = model.get_model_accuracy_multiclass(features=e["features"],
                                            true_classes=e["generator"],
                                            device="cuda:"+str(device))
print(accuracy)

### SVM

In [ ]:
clf.fit(X_train,gen_train).score(X_test,gen_test)

# Saving the model

In [7]:
# torch.save(model.state_dict(),"./checkpoints/multiclass_1000epochs_0.08loss.pt")

# Loading the model

In [8]:
model2 = MultiClassClassifier()
model2.load_state_dict(torch.load("./checkpoints/multiclass_1000epochs_0.08loss.pt"))

<All keys matched successfully>

In [9]:
model2.eval().cuda(device)
with torch.no_grad():
    for e in test_loader:
        pred = torch.argmax(model2(e["features"].cuda(device)),dim=1)
        acc = torch.mean(torch.eq(e["generator"].cuda(device),pred).float()).item()
        print(acc)

0.8808823227882385


# Train on data3 test on SB

In [ ]:
from utils import load_synthbuster_balanced
X_sb, y_sb = load_synthbuster_balanced("../../data/synthbuster_test",
                                       binary_classification=True,
                                       balance_real_fake=True)

In [ ]:
model.get_model_accuracy_binary(torch.Tensor(X_sb).cuda(device),
                                torch.Tensor(y_sb).cuda(device),
                                "cuda:"+str(device))

# Test on OOD

In [6]:
ood = OOD("../../data/ood",load_preprocessed=False,device="cuda:" + str(device))

Processing images from img2img_SD1.5: 100%|██████████| 100/100 [00:06<00:00, 15.16it/s]
Processing images from Photoshop_generativemagnification:  22%|██▏       | 22/100 [00:05<00:20,  3.74it/s]


KeyboardInterrupt: 

In [17]:
# ood.save("../../data/ood.pt")

In [19]:
ood2 = OOD("../../data/ood.pt",load_preprocessed=True)

In [20]:
loader_test = DataLoader(ood2, batch_size=len(ood), shuffle=True)

In [21]:
for e in loader_test:
    model.eval()
    with torch.no_grad():
        accuracy = model.get_model_accuracy_binary(e["features"],e["label"],"cuda:"+str(device))

print(accuracy)

0.5612499713897705
